In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 5-6. Holy トレイト

### 5-6-1. Holy トレイトとは？

#### コード5-38. トレイト型の定義例（`ToTupleStyle`）

In [2]:
abstract type ToTupleStyle end

In [3]:
struct IsAlreadyTuple <: ToTupleStyle end

In [4]:
struct IsScalar <: ToTupleStyle end

In [5]:
struct Splattable <: ToTupleStyle end

#### コード5-39. Holy トレイトによる関数の多重定義例（`tupleit()`）

In [6]:
tupleit(v) = tupleit(ToTupleStyle(v), v)

tupleit (generic function with 1 method)

In [7]:
tupleit(::IsAlreadyTuple, t) = t

tupleit (generic function with 2 methods)

In [8]:
tupleit(::IsScalar, v) = (v,)

tupleit (generic function with 3 methods)

In [9]:
tupleit(::Splattable, xs) = (xs...,)

tupleit (generic function with 4 methods)

#### コード5-40. `ToTupleStyle()` の定義と `tupleit()` の動作例 (1)

In [10]:
ToTupleStyle(::Tuple) = IsAlreadyTuple()

ToTupleStyle

In [11]:
ToTupleStyle(::Number) = IsScalar()

ToTupleStyle

In [12]:
ToTupleStyle(::AbstractArray) = Splattable()

ToTupleStyle

In [13]:
ToTupleStyle(::AbstractString) = IsScalar()
# 文字列は展開可能だが「単一の値」として扱った方が妥当

ToTupleStyle

In [14]:
tupleit((1, :a, '😁'))  # タプルを渡したらそのままタプルを返す

(1, :a, '😁')

In [15]:
tupleit(99.9)  # 数値（スカラー値）を渡したらその値だけからなる要素数1のタプルを返す

(99.9,)

In [16]:
tupleit([3, 1, 4, 1, 6])  # 配列を渡したらその各値からなるタプルを返す

(3, 1, 4, 1, 6)

In [17]:
tupleit(1:5)  # 範囲オブジェクトは AbstractVector の派生型なので

(1, 2, 3, 4, 5)

In [18]:
tupleit([])  # 空の配列なら空のタプルに

()

In [19]:
tupleit("123ABCあいう漢字😁")  # 文字列ならそれだけを単一の値として持つタプルに

("123ABCあいう漢字😁",)

#### コード5-41. `ToTupleStyle()` の定義と `tupleit()` の動作例 (2)

In [20]:
ToTupleStyle(::Any) = IsScalar()  # デフォルトは IsScalar（単一の値）の扱いとする

ToTupleStyle

In [21]:
ToTupleStyle(::NamedTuple) = Splattable()  # NamedTuple は展開可能

ToTupleStyle

In [22]:
ToTupleStyle(::AbstractSet) = Splattable()  # 集合も展開可能

ToTupleStyle

In [23]:
ToTupleStyle(::Ref) = Splattable()  # Ref は展開可能（参照している単一の値が列挙される）

ToTupleStyle

In [24]:
tupleit() = ()  # 引数が空 ⇒ 空のタプル

tupleit (generic function with 5 methods)

In [25]:
tupleit(x, xs...) = (tupleit(x)..., tupleit(xs...)...)
# 引数が2つ以上 ⇒ 再帰呼び出しでそれぞれの要素を `tupleit()`

tupleit (generic function with 6 methods)

In [26]:
tupleit((a=1, b=:b, c='😁'))  # `NamedTuple`

(1, :b, '😁')

In [27]:
tupleit(Set([3, 1, 4, 1, 5, 9, 2, 6, 5, 3]))  # `Set`

(5, 4, 6, 2, 9, 3, 1)

In [28]:
tupleit(Dict(:a => 1, :b => 2))
# 辞書（`Dict`）は展開可能だが、各要素が `:key => ○` のようなペアになるので
# 単一の値として扱うようにする

(Dict(:a => 1, :b => 2),)

In [29]:
tupleit(Ref([1, 5, 3]))
# `Ref` で括ることによって `Splattable` な値もそのまま 
# Scalar のように扱われるようにした

([1, 5, 3],)

In [30]:
tupleit(1, (1, :a, '😁'), 1:5, Ref([3, 1, 4, 1, 6]))  # 複数引数

(1, 1, :a, '😁', 1, 2, 3, 4, 5, [3, 1, 4, 1, 6])

### 5-6-2. 実例：`IndexStyle`

#### コード5-42. `IndexStyle()` の例 (1)（`IndexCartesian` トレイト）

In [31]:
struct SampleSimpleArray{N} <: AbstractArray{NTuple{N, Int}, N}
    size::NTuple{N, Int}
end

In [32]:
IndexStyle(SampleSimpleArray)

IndexCartesian()

#### コード5-43. `IndexCartesian` トレイトの場合のインデクシングの例

In [33]:
Base.size(a::SampleSimpleArray) = a.size

In [34]:
function Base.getindex(a::SampleSimpleArray{N}, idxs::Vararg{Int, N}) where N
    idxs
end

In [35]:
ssa = SampleSimpleArray((3, 4));

In [36]:
ssa[1, 1]

(1, 1)

In [37]:
ssa[2, 3]

(2, 3)

In [38]:
collect(ssa)

3×4 Matrix{Tuple{Int64, Int64}}:
 (1, 1)  (1, 2)  (1, 3)  (1, 4)
 (2, 1)  (2, 2)  (2, 3)  (2, 4)
 (3, 1)  (3, 2)  (3, 3)  (3, 4)

In [39]:
eachindex(ssa)

CartesianIndices((3, 4))

#### コード5-44. `IndexCartesian` トレイトの場合の線形インデックスの例

In [40]:
ssa[1]  # 参考：`fldmod1(1, 3) == (1, 1)`

(1, 1)

In [41]:
ssa[8]  # 参考：`fldmod1(8, 3) == (3, 2)`

(2, 3)

In [42]:
ssa[1:12]

12-element Vector{Tuple{Int64, Int64}}:
 (1, 1)
 (2, 1)
 (3, 1)
 (1, 2)
 (2, 2)
 (3, 2)
 (1, 3)
 (2, 3)
 (3, 3)
 (1, 4)
 (2, 4)
 (3, 4)

#### コード5-45. `IndexStyle` を指定した `eachindex()` の実行例

In [43]:
eachindex(IndexCartesian(), ssa)  # ＝ `eachindex(ssa)`

CartesianIndices((3, 4))

In [44]:
eachindex(IndexLinear(), ssa)

Base.OneTo(12)

In [45]:
[ssa[i] for i in eachindex(IndexLinear(), ssa)]  # 実質 `ssa[1:12]` と同じ

12-element Vector{Tuple{Int64, Int64}}:
 (1, 1)
 (2, 1)
 (3, 1)
 (1, 2)
 (2, 2)
 (3, 2)
 (1, 3)
 (2, 3)
 (3, 3)
 (1, 4)
 (2, 4)
 (3, 4)

#### コード5-46. `IndexStyle()` の例 (2)（`IndexLinear` トレイト）

In [46]:
struct SampleLinearArray{N} <: AbstractArray{Int, N}
    size::NTuple{N, Int}
end

In [47]:
Base.IndexStyle(::Type{<:SampleLinearArray}) = IndexLinear()

In [48]:
IndexStyle(SampleLinearArray)

IndexLinear()

In [49]:
Base.size(a::SampleLinearArray) = a.size

In [50]:
function Base.getindex(a::SampleLinearArray, idx::Int)
    idx
end

#### コード5-47. `IndexLinear` トレイトの場合のインデクシングの例

In [51]:
sla = SampleLinearArray((3, 4));

In [52]:
sla[1]

1

In [53]:
sla[8]

8

In [54]:
collect(sla)

3×4 Matrix{Int64}:
 1  4  7  10
 2  5  8  11
 3  6  9  12

In [55]:
eachindex(sla)

Base.OneTo(12)

#### コード5-48. `IndexLinear` トレイトの場合の直積インデックスの例

In [56]:
sla[1, 1]  # 参考：`(1 - 1) * 3 + 1 == 1`

1

In [57]:
sla[2, 3]  # 参考：`(3 - 1) * 3 + 2 == 8`

8

In [58]:
eachindex(IndexLinear(), sla)  # ＝ `eachindex(sla)`

Base.OneTo(12)

In [59]:
eachindex(IndexCartesian(), sla)

CartesianIndices((3, 4))

In [60]:
[sla[i] for i in eachindex(IndexCartesian(), sla)]  # 実質 `collect(sla)` と同じ

3×4 Matrix{Int64}:
 1  4  7  10
 2  5  8  11
 3  6  9  12